# Проект большие языковые модели

### Подготовка окружения

In [76]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.normalizers import NFD, Lowercase, StripAccents, Sequence
from tokenizers.processors import TemplateProcessing
from transformers import PreTrainedTokenizerFast, LlamaConfig, LlamaForCausalLM, Trainer, TrainingArguments,TrainerCallback, AutoModelForCausalLM, AutoTokenizer
from torch.optim import AdamW 
from datasets import load_dataset
import os
import gc
import torch
import warnings
from torch.utils.data import DataLoader
import regex as re
import numpy as np
import random
from pathlib import Path
from datasets import Dataset
import unicodedata
from collections import Counter
from tqdm.notebook import tqdm
from trl import SFTTrainer
from pprint import pprint

In [27]:
def seed_everything(seed: int):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = True

seed_everything(42)
warnings.filterwarnings('ignore')

# освобождение памяти
def gc_collect():
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize() 

gc_collect()

In [28]:
DATA_DIR = Path("RussianNovels/corpus")
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
VOCAB_SIZE = 3000
CYRILLIC_RE = re.compile(r"^[\p{IsCyrillic}\s0-9.,!?—–:;\"'()«»…\-]+$")
BATCH_SIZE = 2 if DEVICE=='cpu' else 128
MAX_EPOCHS = 12
LEARNING_RATE = 3e-4
WEIGHT_DECAY = 0.01
GRAD_CLIP = 1.0
SAVE_PATH = "best_model.pt"

TEST_PROMPTS = [
    "Все мысли, которые имеют огромные последствия",
    "Сила войска зависит от его духа",
    "Мысль о том, что он принес страдания",
    "Человек сознает себя свободным",
    "Что бы ни случилось, я всегда буду",
    "Любовь мешает смерти",
    "Нет, жизнь не кончена",
    "Всякая мысль, даже самая простая",
    "Война не любезность, а самое гадкое дело",
    "Чтобы жить честно"
] 
MODEL_NAME = "Qwen/Qwen2.5-0.5B"

QUESTIONS = [
    "сколько планет в нашей солнечной системе?",
    "расскажи стих",
    "когда собирать крыжовник?",
    "Как быстро выучить новый язык?"
]

print(f"{DEVICE=}")
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))

DEVICE='cuda'
NVIDIA A100-SXM4-80GB


In [29]:
!git clone https://github.com/JoannaBy/RussianNovels.git
!git clone https://huggingface.co/datasets/d0rj/alpaca-cleaned-ru

fatal: destination path 'RussianNovels' already exists and is not an empty directory.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


fatal: destination path 'alpaca-cleaned-ru' already exists and is not an empty directory.


## Pretrain

### Подготовка корпуска текстов

In [30]:
def collect_texts(root: Path):
    texts = []
    for path in root.rglob("*"):
        if path.suffix in {".txt"}:
            try:
                texts.append(path.read_text(encoding="utf-8"))
            except Exception:
                pass
    return texts

raw_texts = collect_texts(DATA_DIR)
raw_texts = list(set(raw_texts))
print(f"Файлов в корпусе: {len(raw_texts)}")

Файлов в корпусе: 107


In [31]:
def filter_cyrillic_sentences(text: str):
    '''фильтруем предложения с некириллическими символами'''
    sentences = re.split(r"[.!?]+", text)
    return [
        s.strip()
        for s in sentences
        if s.strip() and CYRILLIC_RE.match(s.strip())
    ]

def normalize_punctuation(text: str) -> str:
    '''нормализуемы пунктуацию'''
    text = re.sub(r"[!?]{2,}", r"\1", text)
    text = re.sub(r"\.{3,}", "…", text)
    text = re.sub(r",,{2,}", ",", text)
    text = re.sub(r"-{2,}", "-", text)
    text = re.sub(r"\s+", " ", text)
    text = re.sub(r'[«»]', '"', text)
    return text.strip()    

clean_sentences = []

for text in raw_texts:
    sentences = filter_cyrillic_sentences(text)
    sentences = [normalize_punctuation(s) for s in sentences]
    clean_sentences.extend(sentences)

print(f"Предложений после очистки: {len(clean_sentences)}")

Предложений после очистки: 523962


In [32]:
print("Проверка состава оставшихся спец символов")
def is_letter_or_digit(ch: str) -> bool:
    cat = unicodedata.category(ch)
    return cat.startswith("L") or cat.startswith("N")

counter = Counter()

for sent in clean_sentences:
    for ch in sent:
        if not is_letter_or_digit(ch):
            counter[ch] += 1


non_alnum = counter.most_common()
print(f"symb | freg    | name")
for ch, freq in non_alnum[:30]:
    name = unicodedata.name(ch, "UNKNOWN")
    print(f"'{ch}'  | {freq:>7} | {name}")

Проверка состава оставшихся спец символов


symb | freg    | name
' '  | 5716309 | SPACE
','  |  853248 | COMMA
'-'  |  279089 | HYPHEN-MINUS
'"'  |   49998 | QUOTATION MARK
':'  |   44922 | COLON
';'  |   32563 | SEMICOLON
'–'  |   24641 | EN DASH
')'  |    7897 | RIGHT PARENTHESIS
'('  |    7337 | LEFT PARENTHESIS
'—'  |    4435 | EM DASH
'…'  |    2988 | HORIZONTAL ELLIPSIS
'''  |    1152 | APOSTROPHE


In [33]:
# разбиение на чанки
chunks = []
current = ""

for sent in clean_sentences:
    max_chars = 2000  # ~512 токенов
    if len(current) + len(sent) < max_chars:
        current += " " + sent
    else:
        chunks.append(current.strip())
        current = sent

if current:
    chunks.append(current.strip())

print(f"Чанков: {len(chunks)}")
print(f"чанк  0: {chunks[0]}")
print(f"чанк 10: {chunks[10]}")


Чанков: 20041
чанк  0: Навстречу ко мне выбежал слуга Я потребовал комнату и прибавил: - Смотри же, только чистую, пожалуйста - Уж не побрезгайте, окнами на двор,- умильно глядя на меня, сказал слуга Я заглянул на грязный двор, заставленный различными весьма странными экипажами Под навесом стояли лошади, коровы, бараны На дворе толпились мужики; шум был ужасный Желая хорошенько выспаться после трех ночей, проведенных в телеге, я потребовал комнату непременно с окнами на улицу - Все занято,- отвечал слуга - А налево-то от нас, Архип - раздался мужской голос над нашими головами В окне второго этажа покоились животами на пуховых подушках в ситцевых наволочках старик и старушка - Занято - нехотя отвечал Архип на их замечание - Ну так пятый номер, что опорожнил сегодня купец,- подхватила старушка - Исправник взял под кого-то - грубо крикнул Архип Благодаря заботливости стариков, мне ничего более не оставалось, как попытать счастья в другом трактире - Ты поезжай к немцу, может, у него есть -

### Обучение токенизатора

In [34]:
tokenizer = Tokenizer(BPE(unk_token="[UNK]"))

# Нормализация текста
tokenizer.normalizer = Sequence([NFD(), Lowercase(), StripAccents()])
# разбиение по пробелам 
tokenizer.pre_tokenizer = Whitespace() 
trainer = BpeTrainer(
    vocab_size=VOCAB_SIZE,
    min_frequency=2,
    special_tokens=["[PAD]", "[UNK]", "[BOS]", "[EOS]"] 
    )
tokenizer.train_from_iterator(chunks, trainer)
# (BOS/EOS) 
tokenizer.post_processor = TemplateProcessing(
    single="[BOS] $A [EOS]",
    pair="[BOS] $A [EOS] $B:1 [EOS]:1",
    special_tokens=[
        ("[BOS]", tokenizer.token_to_id("[BOS]")),
        ("[EOS]", tokenizer.token_to_id("[EOS]")), 
        ], 
        )
# Сохранение
tokenizer.save("bpe_ru_3k.json")

In [35]:
encoded = tokenizer.encode("Пример текста для проверки токенизатора")
print(encoded.tokens)
print(encoded.ids)


['[BOS]', 'пример', 'те', 'к', 'ста', 'для', 'про', 'вер', 'ки', 'то', 'ке', 'ни', 'за', 'тора', '[EOS]']
[2, 1903, 80, 31, 123, 308, 106, 170, 100, 58, 220, 70, 81, 1680, 3]


### Сборка датасета

In [36]:

hf_tokenizer = PreTrainedTokenizerFast(
    tokenizer_file="bpe_ru_3k.json",
    bos_token="[BOS]",
    eos_token="[EOS]",
    unk_token="[UNK]",
    pad_token="[PAD]",
)

def tokenize(batch):
    return hf_tokenizer(
        batch["text"],
        truncation=True,
        padding="max_length",
        max_length=512,
    )

def add_labels(batch):
    labels = []
    for ids, mask in zip(batch["input_ids"], batch["attention_mask"]):
        l = ids.copy()
        l = [
            token if m == 1 else -100
            for token, m in zip(l, mask)
        ]
        labels.append(l)

    batch["labels"] = labels
    return batch


dataset = Dataset.from_dict({"text": chunks})

dataset = dataset.map(
    tokenize,
    batched=True,
    remove_columns=["text"],
).map(add_labels, batched=True)

dataset.set_format(type="torch")

Map:   0%|          | 0/20041 [00:00<?, ? examples/s]

Map:   0%|          | 0/20041 [00:00<?, ? examples/s]

### Подготовка модели LlamaForCausalLM

In [37]:
config = LlamaConfig(
    hidden_size=1024,
    intermediate_size=1536,
    num_hidden_layers=16,
    num_attention_heads=16,
    num_key_value_heads=8,
    max_position_embeddings=2048,
    rms_norm_eps=1e-6,
    hidden_act="silu",
    tie_word_embeddings=True,
    attention_bias=False,
    rope_theta=10000.0,
    use_cache=False,     
)
config.vocab_size = max(hf_tokenizer.get_vocab().values()) + 1
config.pad_token_id = hf_tokenizer.pad_token_id
config.bos_token_id = hf_tokenizer.bos_token_id
config.eos_token_id = hf_tokenizer.eos_token_id

model = LlamaForCausalLM(config).to(DEVICE)
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Обучаемых параметров: {num_params:,}")

Обучаемых параметров: 128,934,912


In [38]:
print("Проверим синхронизацию модели и текенизатора")
assert model.config.vocab_size == max(hf_tokenizer.get_vocab().values()) + 1
assert model.config.pad_token_id == hf_tokenizer.pad_token_id
assert model.config.eos_token_id == hf_tokenizer.eos_token_id
print("ок")

Проверим синхронизацию модели и текенизатора
ок


In [39]:
print("Sanity-check начальной модели на инференсе")
model.eval()

prompt = "Привет"
inputs = hf_tokenizer(
    prompt,
    return_tensors="pt",
    return_token_type_ids=False
).to(model.device)

with torch.no_grad():
    out = model.generate(
        **inputs,
        max_new_tokens=30,
        do_sample=False,
        eos_token_id=hf_tokenizer.eos_token_id,
    )

print(hf_tokenizer.decode(out[0], skip_special_tokens=True))


Sanity-check начальной модели на инференсе


при вет кры кры брат брат брат брат брат брат брат брат бары бары бары бары бары бары бары бары бары бары бары бары бары бары бары бары бары бары бары бары


In [40]:
# оптимизатор
def get_optim(model):
    return AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

optimizer = get_optim(model)

In [41]:
print("Сheck сходимости модели при обучении на малом датасете (2 предожения)")
tiny_dataset = dataset.select(range(2))
tiny_loader = DataLoader(tiny_dataset, batch_size=1, shuffle=True)

model.train()

progress_bar = tqdm(
        range(201),
        desc=f"train check",
    )
best_loss = float("inf")

for step in progress_bar:
    batch = next(iter(tiny_loader))
    batch = {k: v.to(model.device) for k, v in batch.items()}

    loss = model(**batch).loss
    progress_bar.set_description(f"train check | loss: {loss:.2f}")
    
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if step> 0 and step % 20 == 0:
        progress_bar.write(f"epoch: {step:>3} loss: {loss.item():.2f}")
        if loss.item() < best_loss:
            best_loss = loss.item()
            torch.save(model.state_dict(), SAVE_PATH)
            progress_bar.write(f"best model saved with {best_loss=:.2f}")

del tiny_dataset, tiny_loader, optimizer
gc_collect()

Сheck сходимости модели при обучении на малом датасете (2 предожения)


train check:   0%|          | 0/201 [00:00<?, ?it/s]

epoch:  20 loss: 5.70
best model saved with best_loss=5.70
epoch:  40 loss: 5.16
best model saved with best_loss=5.16
epoch:  60 loss: 5.05
best model saved with best_loss=5.05
epoch:  80 loss: 2.97
best model saved with best_loss=2.97
epoch: 100 loss: 0.50
best model saved with best_loss=0.50
epoch: 120 loss: 0.04
best model saved with best_loss=0.04
epoch: 140 loss: 0.01
best model saved with best_loss=0.01
epoch: 160 loss: 0.00
best model saved with best_loss=0.00
epoch: 180 loss: 0.00
best model saved with best_loss=0.00
epoch: 200 loss: 0.01


In [42]:
print("Сheck инференса модели после обучения на малом датасете")
def check_inference(model, prompts: list):
    model.eval()
    for pr in prompts:
        inputs = hf_tokenizer(pr, return_tensors="pt").to(model.device)
        inputs.pop("token_type_ids", None)

        out = model.generate(
        **inputs,
        max_new_tokens=50,
        do_sample=True, 
        eos_token_id=hf_tokenizer.eos_token_id,
    )
        print("input:  ",  pr)
        print("output: ", hf_tokenizer.decode(out[0], skip_special_tokens=True))
        print()


check_inference(model, TEST_PROMPTS[:2])

Сheck инференса модели после обучения на малом датасете
input:   Все мысли, которые имеют огромные последствия
output:  все мысли , которые име ют огром ные послед ствия на скои па вече вы объя вы встречу меня , да ,- да и то на двор , не по знает я делать , да меня , не муж вы знает слу ге : - уж не зная , ар хи п , зя , не отвечал ар хи п

input:   Сила войска зависит от его духа
output:  сила вои ска зави си т от его ду ха ар хи п на от рез объя вил , что нет ,- разве не вы будет ли кто к вече ру , да и то бог знает я стоял в недо уме нии , не зная , что делать , как вдруг стари чок крикнул слу ге : - ар



### Обучение модели LlamaForCausalLM

In [43]:
# Callback для валидации процесса обучения
class PromptCallback(TrainerCallback):
    def __init__(self, tokenizer, prompts, max_new_tokens=50):
        self.tokenizer = tokenizer
        self.prompts = prompts
        self.max_new_tokens = max_new_tokens

    def _one_answer(self, model, prompt):
        inputs = self.tokenizer(prompt, return_tensors="pt").to(model.device)
        inputs.pop("token_type_ids", None)
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=self.max_new_tokens,
                do_sample=True, 
                eos_token_id=hf_tokenizer.eos_token_id,
            )
        return self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        

    def on_evaluate(self, args, state, control, **kwargs):
        model = kwargs.get("model")
        if model is None:
            return
        model.eval()
        print("="*20, f"on step {state.global_step}", "="*20)
        for prompt in self.prompts[:3]:
            decoded = self._one_answer(model, prompt)
            print("input:  ",  prompt)
            print("output: ", decoded)
            print()    
            
    def on_train_end(self, args, state, control, **kwargs):
        model = kwargs.get("model")
        if model is None:
            return
        model.eval()
        print("="*20, f"ON TRAIN END", "="*20)        
        for prompt in self.prompts:
            decoded = self._one_answer(model, prompt)
            print("input:  ",  prompt)
            print("output: ", decoded)
            print()    


In [45]:
# обучение модели

split_dataset = dataset.train_test_split(test_size=0.01)
train_dataset = split_dataset['train']
val_dataset = split_dataset['test']

training_args = TrainingArguments(
    output_dir="./checkpoints",
    num_train_epochs=MAX_EPOCHS,
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=4,  # 4 × 128 
    weight_decay=WEIGHT_DECAY,
    eval_strategy="steps",
    save_strategy="no",
    eval_steps=1 if DEVICE=='cpu' else 50,
    logging_steps=1 if DEVICE=='cpu' else 50,
    fp16=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=hf_tokenizer,
    callbacks=[
        PromptCallback(
            tokenizer=hf_tokenizer,
            prompts=TEST_PROMPTS
        )
    ]
)
train_output = trainer.train()

Step,Training Loss,Validation Loss
50,5.428000,5.102242
100,4.841100,4.618660
150,4.387800,4.284968
200,4.075100,4.125978
250,3.850000,4.043071
300,3.664600,4.016393
350,3.498600,4.021312
400,3.322900,4.087390
450,3.195300,4.124145


==================== on step 50 ====================
input:   Все мысли, которые имеют огромные последствия
output:  все мысли , которые име ют огром ные послед ствия х , все более вели чи ми ся и даже с тобои от стра ст нои , чтобы с нима ть , которыи в этом отноше ниях в самом деле что и то чка , да ст рые , к ли ще нием и без сомне ния , на ле

input:   Сила войска зависит от его духа
output:  сила вои ска зави си т от его ду ха : он только ты не види м их , раз би р ались - по тря ше и - про па кли - с , я знаю ты вать , - сказал ему , нет , пере бира ющии тебя , при ну че ли на , о хва тил

input:   Мысль о том, что он принес страдания
output:  мысль о том , что он при нес страда ния не могла про шеп тал с собои , но он не только в эту минуту не при этом ; но если б рю я в чем время не которые , не под ходил до этого у вели чи ка , а там да ве , не которые по красне ли

==================== on step 100 ====================
input:   Все мысли, которые имеют огромные последствия
out

In [46]:
del split_dataset, train_dataset, val_dataset, model, trainer
gc_collect()

## Post-train SFT

### Подготовка модели QWEN

In [111]:
tokenizer_qwen = AutoTokenizer.from_pretrained(
    MODEL_NAME, use_fast=True
)

model_qwen = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float32,
    device_map="auto",
)
#model_qwen = model_qwen.cuda().half()
model_qwen.config.use_cache = False

### Ответы сырой модели

In [99]:
def check_outputs(model, tokenizer, questtions):
    def _generate_answer(model, question):
        mess = [
            {"role": "user",
            "content": question}
            ]
        inputs = tokenizer.apply_chat_template(
            mess,
            return_tensors="pt",
            pad_token_id=tokenizer.pad_token_id
        ).to(model_qwen.device)

        output = model_qwen.generate(
            inputs,
            max_new_tokens=50,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id
        )
        return tokenizer.decode(output[0], skip_special_tokens=True)

    for idx, q in enumerate(questtions):
        print(f"Model Input {idx+1}:")
        print(q)
        print(f"Model Output {idx+1}:")
        print(_generate_answer(model, q))
        print("="*40)

check_outputs(model_qwen, tokenizer_qwen, QUESTIONS[:3])        

Model Input 1:
сколько планет в нашей солнечной системе?
Model Output 1:
system
You are a helpful assistant.
user
сколько планет в нашей солнечной системе?
onnementi
сколько планет в нашей солнечной системе? libertine i
сколько планет в нашей солнечной системе? libertine
сколько планет в нашей солнечной системе? libertine
сколько планет в
Model Input 2:
расскажи стих
Model Output 2:
system
You are a helpful assistant.
user
расскажи стих
itative и критично
itative это
itative это
itative это тонкий
itative это тонкий

itative это тонкий
itative это тонкий
Model Input 3:
когда собирать крыжовник?
Model Output 3:
system
You are a helpful assistant.
user
когда собирать крыжовник?
tica
Самые большие брексы у нас о них можно держать как 80 см.. сколько? nrw
tica
за сколько межкабельный шиповник
tica
60


### Подготовка датасета

In [100]:
ds = load_dataset("d0rj/alpaca-cleaned-ru", split="train")
len(ds)

51760

In [101]:
def example_to_message(example):
    return {
        "messages": [
            {'role': 'user', 'content': example['instruction']},
            {'role': 'assistant', 'content': example['output']}
        ]
    }

ds_chat = ds.map(
    example_to_message,
    batched=False,
    remove_columns=['input', 'instruction', 'output', ],
).map(
    lambda x: {
        'text': tokenizer_qwen.apply_chat_template(x['messages'],
         tokenize=False)
         },
        remove_columns=['messages'],
    )
pprint(ds_chat[0])

{'text': '<|im_start|>system\n'
         'You are a helpful assistant.<|im_end|>\n'
         '<|im_start|>user\n'
         'Дайте три совета, как оставаться здоровым.<|im_end|>\n'
         '<|im_start|>assistant\n'
         '1. Соблюдайте сбалансированную и питательную диету. Убедитесь, что в '
         'ваш рацион входят разнообразные фрукты и овощи, нежирный белок, '
         'цельнозерновые продукты и полезные жиры. Это помогает обеспечить ваш '
         'организм необходимыми питательными веществами для оптимального '
         'функционирования и может помочь предотвратить хронические '
         'заболевания.\n'
         '\n'
         '2. Занимайтесь регулярной физической активностью. Упражнения имеют '
         'решающее значение для поддержания крепких костей, мышц и здоровья '
         'сердечно-сосудистой системы. Старайтесь уделять не менее 150 минут '
         'умеренным аэробным упражнениям или 75 минут интенсивным упражнениям '
         'каждую неделю.\n'
         '\n'
    

### Обучение модели QWEN

In [ ]:
torch.cuda.empty_cache()

split_ds_chat = ds_chat.train_test_split(test_size=0.01)
train_dataset = split_ds_chat['train']
val_dataset = split_ds_chat['test']

In [ ]:
class ClearCUDACallback(TrainerCallback):
    def on_evaluate(self, args, state, control, **kwargs):
        torch.cuda.empty_cache()

def formatting_func(example):
    return example["text"]

training_args_qwen = TrainingArguments(
    output_dir="./checkpoints_qwen",
    num_train_epochs=3,
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=3,
    weight_decay=WEIGHT_DECAY,
    eval_strategy="steps",
    save_strategy="no",
    eval_steps=1 if DEVICE=='cpu' else 1000,
    logging_steps=1 if DEVICE=='cpu' else 1000,
    fp16=True,
    bf16=False,  
    report_to="none",
)

trainer = SFTTrainer(
    model_qwen,
    args=training_args_qwen,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    formatting_func=formatting_func, 
    processing_class=tokenizer_qwen,  
    callbacks=[ClearCUDACallback()],
)

train_output = trainer.train()

Step,Training Loss,Validation Loss,Entropy,Num Tokens,Mean Token Accuracy
1000,1.849400,1.756764,1.833235,6507415.000000,0.622389
2000,1.587900,1.555425,1.571363,12936854.000000,0.655171


In [97]:
del model_qwen, split_ds_chat,  ds_chat, train_dataset, val_dataset 
gc_collect()

In [92]:
print("Model dtype:", next(model_qwen.parameters()).dtype)


Model dtype: torch.float16


In [104]:
batch = next(iter(trainer.get_train_dataloader()))
print(batch.keys())

AttributeError: `AcceleratorState` object has no attribute `distributed_type`. This happens if `AcceleratorState._reset_state()` was called and an `Accelerator` or `PartialState` was not reinitialized.